In [ ]:
import torch
from torchvision import transforms
from torchvision.datasets import MNIST 
from torch.autograd import gradcheck
import numpy as np
import matplotlib.pyplot as plt

# **Classification**
In this part we will explore the performance of using different activation units to train on a subset of MNIST dataset

First, we load the dataset and turn it into numpy arrays

In [ ]:
mnist_train_set = MNIST("Data", download = True, train = True)
mnist_test_set = MNIST("Data", download = True, train = False)
mnist_trainX = np.array(mnist_train_set.data.numpy())
mnist_trainY = np.array(mnist_train_set.targets.numpy())
mnist_testX = np.array(mnist_test_set.data.numpy())
mnist_testY = np.array(mnist_test_set.targets.numpy())
mnist_trainX = mnist_trainX.reshape((mnist_trainX.shape[0], -1))
mnist_testX = mnist_testX.reshape((mnist_testX.shape[0], -1))


### (a) Visualization

Visualize a mnist data and its label. 

In [ ]:
############################################################### TODO A

############################################################### TODO A

In the cell below, split the validation set with fraction 0.2

### (b) Split training set and the validation set

Split training set and the validation set

In [ ]:
############################################# TODO B
# Please Fill in the cell to implement X_train, Y_train, X_valid, Y_valid, X_test, and Y_test in numpy array
# Also provide the number of data in n_train, n_valid, and n_test

#############################################

# Turn the numpy array to pytorch tensor

X_train = torch.Tensor(X_train)
Y_train = torch.from_numpy(Y_train)
# split the validation set
X_valid = torch.Tensor(X_valid)
Y_valid = torch.from_numpy(Y_valid)
# renaming the test data
X_test = torch.Tensor(X_test)
Y_test = torch.from_numpy(Y_test)

print("Number of training data: {}".format(n_train))
print("Number of validation data: {}".format(n_valid))
print("Number of test data: {}".format(n_test))

### **(c) Implementing activation functions: Mish and Swish**
Pytorch provides many activation units under torch.nn.Module.

We can just use torch.nn.{Module Name} to instantiates the module. [Here is the reference of the official documentation on the types of activation functions and how to use them](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity)

Moreover, we can also define new activation units.

In this problem, we will implement Mish and Swish activation function

The original paper for Mish : https://arxiv.org/abs/1908.08681

The original paper for Swish: https://arxiv.org/abs/1710.05941

The Mish function is defined as:

$f_{Mish}(x) = x(tanh(ln(1+e^x)))$

And the Swish function is defined as: 

$f_{Swish}(x) = x(sigmoid(\beta x)) = \frac{x}{( 1 + e^{\beta x})}$, where $\beta$ is a learnable weight 

Implement the two function by completing forward and backward(gradient) method.
You can use torch.gradcheck to see if your gradient is computed correctly. The torch.gradcheck compares the analytical gradients to numerical gradients, in which the analytical gradients are obtained by the backward methods and the numerical gradients are obtained by introducing small difference to the input.

If your implementation is corrent. The gradcheck function will return True.
For more implemenation details, you can see the official note from pytorch: https://pytorch.org/docs/stable/notes/extending.html

In [ ]:
# This is an example of how to implement an autograd function in pytorch
# the forward function compute the output of the function 
# the backward function computes the gradient with respect to the input (and other parameters like weights)
# For activation function, this can be easily done by chain rule.
# As the example, the gradient w.r.t input equals the gradient w.r.t its output multiplied 
#    by the gradient of its output w.r.t its input 
# The ctx.saved_tensors can save the input and some tempory results from forward method
# In the backward method, call ctx.saved_tensors to load the input and tempory results
class SigmoidFunction(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        output = 1.0/ ( 1 + torch.exp(-input))
        ctx.save_for_backward(output)   # save for backward function
        return output
    
    @staticmethod
    def backward(ctx, grad_output):
        output, = ctx.saved_tensors
        grad_input = output * (1 - output) * grad_output
        return grad_input

class MishFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ################################################### TODO C_1

        ###################################################
        return output

    @staticmethod
    def backward(ctx, grad_output):
        ################################################### TODO C_2

        ###################################################
        return grad_input    
    
class SwishFunction(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input, beta):
        ################################################### TODO C_3

        ################################################### 
        return output

    @staticmethod
    def backward(ctx, grad_output):
        # The beta in swish function is learnable. Therefore you have to also compute the gradients w.r.t beta
        ################################################### TODO C_4

        ################################################### TODO
        return grad_input, grad_beta
    
sigmoid = SigmoidFunction.apply
mish = MishFunction.apply
swish = SwishFunction.apply


class Sigmoid(torch.nn.Module):
    
    def __init__(self):
        super(MySigmoid, self).__init__()

    def forward(self, input):
        return sigmoid(input)

class Mish(torch.nn.Module):
    
    def __init__(self):
        super(Mish, self).__init__()

    def forward(self, input):
        return sigmoid(input)
    
class Swish(torch.nn.Module):
    
    def __init__(self):
        super(Swish, self).__init__()
        self.beta = torch.nn.Parameter(torch.Tensor(1))
        self.beta.data.uniform_(-0.1, 0.1)
        
    def forward(self, input):
        return swish(input, self.beta)

input = (torch.randn(50,50,dtype=torch.double,requires_grad=True))
test_result = gradcheck(sigmoid, input, eps=1e-6, atol=1e-4)
print("Gradient check for sigmoid function: ", test_result)
input = (torch.randn(50,50,dtype=torch.double,requires_grad=True))
test_result = gradcheck(mish, input, eps=1e-6, atol=1e-4)
print("Gradient check for mish function: ", test_result)
input = (torch.randn(50,50,dtype=torch.double,requires_grad=True), torch.randn(50,50,dtype=torch.double,requires_grad=True))
test_result = gradcheck(swish, input, eps=1e-6, atol=1e-4)
print("Gradient check for swish function: ", test_result)

### (d) Training neural networks using different activation function



In [ ]:
D_in = mnist_trainX.shape[1]
D_out = 10

def training(activation_module, D_hidden_in, D_hidden_out, x, y, learning_rate, epochs):
    
    model = torch.nn.Sequential(
        torch.nn.Linear(D_in, D_hidden_in),
        activation_module(),
        torch.nn.Linear(D_hidden_in, D_hidden_out),
        activation_module(),
        torch.nn.Linear(D_hidden_out, D_out),
    )
    
    loss_train_his = []
    loss_valid_his = []
    acc_train_his = []
    acc_valid_his = []

    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    for i in range(epochs):
        
        if i%10 == 9:
            with torch.no_grad():
                model.eval()
                loss_valid, acc_valid = test(model, X_valid, Y_valid)
                loss_valid_his.append(loss_valid)
                acc_valid_his.append(acc_valid)
                loss_train, acc_train = test(model, x, y)
                loss_train_his.append(loss_train)
                acc_train_his.append(acc_train)
                print(i, "loss = ", loss_train, "acc = ", acc_train, "valid: ", (loss_valid, acc_valid))
        model.train()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Training complete")
    return model, loss_train_his, loss_valid_his, acc_train_his, acc_valid_his

def test(model, X_test, Y_test):
    
    y_pred = model(X_test)
    loss_fn = torch.nn.CrossEntropyLoss()
    loss = loss_fn(y_pred, Y_test)
    _, label = torch.max(y_pred, 1)
    return loss.item(), (torch.count_nonzero(label == Y_test)/ Y_test.shape[0]).item()


Using the above training function, train the classifier using different activation function including ReLU, Sigmoid, LeakyReLU, and Tanh, Swish, and Mish.

Record the returned losses and accuracy in the array losses and accs.

First, using the same learning rate 0.005 for all activation function

In [ ]:
epochs = 1000
learning_rate = 5e-3
activation_units = [torch.nn.ReLU, torch.nn.Sigmoid, torch.nn.LeakyReLU, torch.nn.Tanh, Mish, Swish]
activation_strs = ["ReLU", "Sigmoid", "LeakyReLU", "Tanh",  "Mish", "Swish"]
learning_rate = [5e-3, 5e-3, 5e-3, 5e-3, 5e-3, 5e-3]
models = []
losses = np.zeros((len(activation_units), 2, int(epochs/10)))
accs = np.zeros((len(activation_units), 2, int(epochs/10)))


for i, act in enumerate(activation_units):
    print("Training with {} activation units".format(activation_strs[i]))
# Call the training function to get the model, loss and accuracy
    model, loss_train_his, loss_valid_his, acc_train_his, acc_valid_his = training(act, 
                200, 64, X_train, Y_train, learning_rate[i], epochs)
    models.append(model)
    losses[i,0,:] = np.array(loss_train_his)
    losses[i,1,:] = np.array(loss_valid_his)
    accs[i,0,:] = np.array(acc_train_his)
    accs[i,1,:] = np.array(acc_valid_his)

Visualize the change of loss and accuracy versus epochs.

You should have four plots (1) training loss (2) validation loss (3) training accuracy (4) validation accuracy

Observe the loss and accuracy.

_Your Observation:_

We can see that the ReLU and LeakyReLU have similar loss during training, and their loss have converged. Their performance are the best compared with other activation functions.  For Tanh, and sigmoid activation units, the loss do decrease but have not converged, which indicate that we should slightly increase the learning rate or increase the number of epochs. For Mish activation function 

In [ ]:
x_range = range(9, epochs, 10)
for i, act in enumerate(activation_units):
    plt.plot(x_range, losses[i,0], label = activation_strs[i])
plt.xlabel("Epochs")
plt.ylabel("Training Losses")
plt.yscale("log")
plt.legend()
plt.show()

for i, act in enumerate(activation_units):
    plt.plot(x_range, losses[i,1], label = activation_strs[i])
plt.xlabel("Epochs")
plt.ylabel("Validation Losses")
plt.yscale("log")
plt.legend()
plt.show()

for i, act in enumerate(activation_units):
    plt.plot(x_range, accs[i,0], label = activation_strs[i])
plt.xlabel("Epochs")
plt.ylabel("Training Accuracy")
plt.yscale("log")
plt.legend()
plt.show()

for i, act in enumerate(activation_units):
    plt.plot(x_range, accs[i,1], label = activation_strs[i])
plt.xlabel("Epochs")
plt.ylabel("Validation Accuracy")
plt.yscale("log")
plt.legend()
plt.show()

### **(e) Grid search of learning rate for different activation functions**
You should have noticed that some activation units did not converge.

This means that the learning rate may be too low for those activation units.

Adjust the learning rates for those activation units, and try to optimize the learning rate for each activation unit.

In [ ]:
num_lr = 8
epochs = 1000
losses = np.zeros((len(activation_units), 2, num_lr, int(epochs/10)))
accs = np.zeros((len(activation_units), 2, num_lr, int(epochs/10)))

###################################################### TODO E

######################################################

Observe which learning rate is best for each activation function.
Then compare these activation function result.

This may take about one hours to train all models if you use CPU training.

In [ ]:
x_range = range(9, epochs, 10)
for i, act in enumerate(activation_units):
    for j, lr in enumerate(learning_rates[i]):
        plt.plot(x_range, np.clip(losses[i,0,j], a_min = None, a_max = 100), label = "lr = {:1.4f}".format(lr))
    plt.xlabel("Epochs")
    plt.ylabel("Training Losses")
    #plt.ylim(0, 5)
    plt.yscale("log")
    plt.legend()
    plt.title(activation_strs[i])
    plt.show()

for i, act in enumerate(activation_units):
    for j, lr in enumerate(learning_rates[i]):
        plt.plot(x_range, np.clip(losses[i,1,j], a_min = None, a_max = 100), label = "lr = {:1.4f}".format(lr))
    plt.xlabel("Epochs")
    plt.ylabel("Validation Losses")
    #plt.ylim(0, 5)
    plt.yscale("log")
    plt.legend()
    plt.title(activation_strs[i])
    plt.show()

for i, act in enumerate(activation_units):
    for j, lr in enumerate(learning_rates[i]):
        plt.plot(x_range, accs[i,0,j], label = "lr = {:1.4f}".format(lr))
    plt.xlabel("Epochs")
    plt.ylabel("Training Accuracy")
    #plt.ylim(0, 5)
    plt.yscale("log")
    plt.legend()
    plt.title(activation_strs[i])
    plt.show()

for i, act in enumerate(activation_units):
    for j, lr in enumerate(learning_rates[i]):
        plt.plot(x_range, accs[i,1,j], label = "lr = {:1.4f}".format(lr))
    plt.xlabel("Epochs")
    plt.ylabel("Validation Accuracy")
    #plt.ylim(0, 5)
    plt.yscale("log")
    plt.legend()
    plt.title(activation_strs[i])
    plt.show()

### **(f) Training using the best learning rate**
Using the best learning rate to train on the complete training set.

Compare the loss and accuracy during training.

Then report the test accuracy for each activation. 

In [ ]:
X_trainall = torch.Tensor(mnist_subsetX)
Y_trainall = torch.from_numpy(mnist_subsetY)

epochs = 1000

models = []
losses = np.zeros((len(activation_units), 2, int(epochs/10)))
accs = np.zeros((len(activation_units), 2, int(epochs/10)))
###################################################### TODO F

######################################################

In [ ]:
x_range = range(9, epochs, 10)
for i, act in enumerate(activation_units):
    plt.plot(x_range, losses[i,0], label = activation_strs[i])
plt.xlabel("Epochs")
plt.ylabel("Training Losses")
plt.yscale("log")
plt.legend()
plt.show()

for i, act in enumerate(activation_units):
    plt.plot(x_range, accs[i,0], label = activation_strs[i])
plt.xlabel("Epochs")
plt.ylabel("Training Accuracy")
#plt.yscale("log")
plt.legend()
plt.show()


In [ ]:
# Report the accuracy on test set
###################################################### TODO F

###################################################### TODO